In [1]:
from pathlib import Path
import sys

ROOT = Path.cwd().resolve()
if not (ROOT / "classes").exists():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))

from classes.data_loader import load_game_data
from classes.pokemon import Pokemon, Stats
from classes.type import Type
from classes.move import Move
from classes.item import Item
from classes.ability import Ability


In [2]:
game = load_game_data()
type_chart = game.type_chart


def slug(text: str) -> str:
    return "".join(ch for ch in text.lower() if ch.isalnum())


abilities = {slug(a.name): a for a in game.abilities.values()}
items = {slug(i.name): i for i in game.items.values()}
moves = {slug(m.name): m for m in game.moves.values()}
species = {slug(s.name): s for s in game.species.values()}

nature_modifiers = {
    "timid": {"spe": 1.1, "atk": 0.9},
    "adamant": {"atk": 1.1, "spa": 0.9},
    "jolly": {"spe": 1.1, "spa": 0.9},
}

stat_key_map = {
    "hp": "hp",
    "atk": "atk",
    "attack": "atk",
    "def": "defense",
    "defense": "defense",
    "spa": "spa",
    "spatk": "spa",
    "spattack": "spa",
    "spatt": "spa",
    "spd": "spd",
    "spdef": "spd",
    "spdefense": "spd",
    "spe": "spe",
    "speed": "spe",
}


def parse_spread(line: str) -> Stats:
    stats = {"hp": 0, "atk": 0, "defense": 0, "spa": 0, "spd": 0, "spe": 0}
    data = line.split(":", 1)[1]
    for chunk in data.split("/"):
        chunk = chunk.strip()
        if not chunk:
            continue
        parts = chunk.split()
        value = int(parts[0])
        key = slug("".join(parts[1:]))
        normalized = stat_key_map.get(key)
        if not normalized:
            raise ValueError(f"Unknown stat key in spread: {key}")
        stats[normalized] = value
    return Stats(stats["hp"], stats["atk"], stats["defense"], stats["spa"], stats["spd"], stats["spe"])


def parse_block(block: str) -> Pokemon:
    lines = [ln.strip() for ln in block.strip().splitlines() if ln.strip()]
    header = lines[0]
    if "@" in header:
        mon_name, item_name = map(str.strip, header.split("@", 1))
    else:
        mon_name, item_name = header, ""

    species_obj = species[slug(mon_name)]
    item_obj = items.get(slug(item_name)) if item_name else None

    ability_line = next((ln for ln in lines if ln.lower().startswith("ability:")), None)
    ability_obj = species_obj.abilities[0]
    if ability_line:
        ability_key = slug(ability_line.split(":", 1)[1].strip())
        ability_obj = abilities.get(ability_key, ability_obj)

    tera_line = next((ln for ln in lines if ln.lower().startswith("tera type:")), None)
    tera_type = type_chart.resolve(tera_line.split(":", 1)[1].strip()) if tera_line else None

    level_line = next((ln for ln in lines if ln.lower().startswith("level:")), None)
    level = int(level_line.split(":", 1)[1]) if level_line else 50

    ev_line = next((ln for ln in lines if ln.lower().startswith("evs:")), None)
    evs = parse_spread(ev_line) if ev_line else Stats(0, 0, 0, 0, 0, 0)

    iv_line = next((ln for ln in lines if ln.lower().startswith("ivs:")), None)
    ivs = parse_spread(iv_line) if iv_line else Stats(31, 31, 31, 31, 31, 31)

    nature_line = next((ln for ln in lines if " nature" in ln.lower()), None)
    nature_key = slug(nature_line.split()[0]) if nature_line else ""
    nature_mod = nature_modifiers.get(nature_key, {})

    move_lines = [ln for ln in lines if ln.startswith("-")]
    move_objs = [moves[slug(ln.lstrip("- "))] for ln in move_lines]

    return Pokemon(
        species=species_obj,
        level=level,
        ability=ability_obj,
        item=item_obj,
        moves=move_objs,
        evs=evs,
        ivs=ivs,
        nature_modifiers=nature_mod,
        tera_type=tera_type,
    )


team_text = Path("../data/import/battle_team_a.txt").read_text()
blocks = [b for b in team_text.split("\n\n") if b.strip()]
team = [parse_block(block) for block in blocks]

for mon in team:
    tera = mon.tera_type.name if mon.tera_type else None
    move_names = ", ".join(move.name for move in mon.moves)
    print(f"{mon.species.name} @ {mon.item.name if mon.item else 'None'} | Ability: {mon.ability.name} | Tera: {tera} | Moves: {move_names}")

team_summary = [
    {
        "name": mon.species.name,
        "item": mon.item.name if mon.item else None,
        "ability": mon.ability.name,
        "tera": mon.tera_type.name if mon.tera_type else None,
        "evs": mon.evs.to_dict(),
        "ivs": mon.ivs.to_dict(),
        "nature_modifiers": mon.nature_modifiers,
        "moves": [mv.name for mv in mon.moves],
    }
    for mon in team
]

team_summary

Gholdengo @ Life Orb | Ability: Good as Gold | Tera: steel | Moves: Make It Rain, Shadow Ball, Nasty Plot, Protect
Kilowattrel @ Focus Sash | Ability: Wind Power | Tera: electric | Moves: Tailwind, Thunderbolt, Air Slash, Protect
Rillaboom @ Assault Vest | Ability: Grassy Surge | Tera: fire | Moves: Grassy Glide, Wood Hammer, Fake Out, U-turn
Kingambit @ Black Glasses | Ability: Supreme Overlord | Tera: fairy | Moves: Kowtow Cleave, Sucker Punch, Iron Head, Protect
Ninetales-Alola @ Light Clay | Ability: Snow Warning | Tera: water | Moves: Aurora Veil, Freeze-Dry, Moonblast, Protect
Basculegion-F @ Choice Band | Ability: Adaptability | Tera: ghost | Moves: Wave Crash, Last Respects, Aqua Jet, Flip Turn


[{'name': 'Gholdengo',
  'item': 'Life Orb',
  'ability': 'Good as Gold',
  'tera': 'steel',
  'evs': {'hp': 52, 'atk': 0, 'defense': 4, 'spa': 196, 'spd': 4, 'spe': 252},
  'ivs': {'hp': 31, 'atk': 0, 'defense': 31, 'spa': 31, 'spd': 31, 'spe': 31},
  'nature_modifiers': {'spe': 1.1, 'atk': 0.9},
  'moves': ['Make It Rain', 'Shadow Ball', 'Nasty Plot', 'Protect']},
 {'name': 'Kilowattrel',
  'item': 'Focus Sash',
  'ability': 'Wind Power',
  'tera': 'electric',
  'evs': {'hp': 0, 'atk': 0, 'defense': 4, 'spa': 252, 'spd': 0, 'spe': 252},
  'ivs': {'hp': 31, 'atk': 0, 'defense': 31, 'spa': 31, 'spd': 31, 'spe': 31},
  'nature_modifiers': {'spe': 1.1, 'atk': 0.9},
  'moves': ['Tailwind', 'Thunderbolt', 'Air Slash', 'Protect']},
 {'name': 'Rillaboom',
  'item': 'Assault Vest',
  'ability': 'Grassy Surge',
  'tera': 'fire',
  'evs': {'hp': 252, 'atk': 196, 'defense': 0, 'spa': 0, 'spd': 60, 'spe': 0},
  'ivs': {'hp': 31, 'atk': 31, 'defense': 31, 'spa': 31, 'spd': 31, 'spe': 31},
  'natur

In [8]:
team_text = Path("../data/import/battle_team_a.txt").read_text()
blocks = [b for b in team_text.split("\n\n") if b.strip()]
team_a = [parse_block(block) for block in blocks]

team_text = Path("../data/import/battle_team_b.txt").read_text()
blocks = [b for b in team_text.split("\n\n") if b.strip()]
team_b = [parse_block(block) for block in blocks]

for mon in team_a+team_b:
    # tera = mon.tera_type.name if mon.tera_type else None
    # move_names = ", ".join(move.name for move in mon.moves)
    # print(mon.species.name)
    # print(mon.item.name)
    # print(mon.ability.name)
    for mv in mon.moves:
        print(mv.name)

Make It Rain
Shadow Ball
Nasty Plot
Protect
Tailwind
Thunderbolt
Air Slash
Protect
Grassy Glide
Wood Hammer
Fake Out
U-turn
Kowtow Cleave
Sucker Punch
Iron Head
Protect
Aurora Veil
Freeze-Dry
Moonblast
Protect
Wave Crash
Last Respects
Aqua Jet
Flip Turn
Heat Wave
Power Gem
Earth Power
Protect
Aqua Jet
Hyper Voice
Moonblast
Encore
Trick Room
Recover
Ice Beam
Thunderbolt
Bitter Blade
Shadow Sneak
Close Combat
Protect
Spore
Rage Powder
Clear Smog
Protect
Fake Out
Parting Shot
Knock Off
Flare Blitz
